In [ ]:
#Importing Libraries
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,Lambda
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
#Defining the image size as VGG16 has got trained in image size of (224,224)
IMAGE_SIZE=[224,224]

In [ ]:
#Saving the data location path in variables
df_train='../input/cat-and-dog/training_set'
df_test='.../input/cat-and-dog/test_set'

In [ ]:
#Importing the VGG16 model and removing the last layer "include_top=False"
#We have to remove the last layer because we need to predict for 2 classes and VGG16 is 1000 class classification model
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [ ]:
#Setting the weight updation of all layers as False as we will not train them
for layers in vgg.layers:
  layers.trainable=False

In [ ]:
#Flattening the output of VGG16 model
x=Flatten()(vgg.output)

In [ ]:
#Adding a dense layer for prediction
prediction=Dense(2,activation='sigmoid')(x)

In [ ]:
#Instantiating a model
model=Model(inputs=vgg.input,outputs=prediction)

In [ ]:
#Understanding the parameters of model
model.summary()

In [ ]:
#Compiling the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#setting the parameters for train dataset
train_data=ImageDataGenerator(rescale=(1/255.0),
                              zoom_range=0.2,
                              rotation_range=0.2,
                              horizontal_flip=True,
                              shear_range=0.1)

In [ ]:
#setting the parameters for test dataset 
test_data=ImageDataGenerator(rescale=(1/255.0))

In [ ]:
#Creating more images using ImageDataGenerator class
training=train_data.flow_from_directory(directory='../input/cat-and-dog/training_set/training_set', target_size=(224,224), 
                                           classes=['cats', 'dogs'], batch_size=10, shuffle=False)
testing=test_data.flow_from_directory(directory= '../input/cat-and-dog/test_set/test_set', target_size=(224,224), classes=['cats', 'dogs'], 
                                       batch_size=10, shuffle=False)

In [ ]:
#fitting the model
answer=model.fit(training,validation_data=testing,epochs=10,steps_per_epoch=len(training),validation_steps=len(testing))

In [ ]:
#Checking the keys stored in the variable answer
answer.history.keys

In [ ]:
#Plotting the training accuracy and validation accuracy graphs
plt.plot(answer.history['val_accuracy'],label=['Validation Accuracy'])
plt.plot(answer.history['accuracy'],label=['Training Accuracy'])
plt.legend()

In [ ]:
#plotting the training and validation loss graphs
plt.plot(answer.history['val_loss'],label=['Validation Loss'])
plt.plot(answer.history['loss'],label=['Training Loss'])
plt.legend()

In [ ]:
#predicting using the model on the test data
y_pred=model.predict(testing)

In [ ]:
#Storing the true values
y_test=testing.classes

In [ ]:
#Getting the classification report
report =classification_report(y_true=testing.classes, y_pred=np.argmax(y_pred, axis=-1))
print(report)

In [ ]:
#Plotting the confusion matrix
cm = confusion_matrix(y_true=testing.classes, y_pred=np.argmax(y_pred, axis=-1))
sns.heatmap(cm,annot=True)
print(cm)

**Predicting using the model**

In [ ]:
dat = '../input/cat-and-dog/test_set/test_set/cats/cat.4009.jpg' # For cat images
dat1 = '../input/cat-and-dog/test_set/test_set/dogs/dog.4006.jpg' # For Dog images

import numpy as np
from keras.preprocessing import image
test1 = image.load_img(dat1, target_size = (224, 224)) # Change when directory change 

test= image.load_img(dat1, target_size = (224, 224)) # Change when directory change 
test= image.img_to_array(test)
test = np.expand_dims(test, axis = 0)
result = model.predict(test)
training.class_indices
if result[0][0] == 1:
  prediction = 'cat'
else:
  prediction = 'dog'

print(prediction)
print('\n')
plt.imshow(test1)